In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('reviews.csv')

In [ ]:
df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2384,25218143,2015-01-09,14385014,Ivan,it's a wonderful trip experience. I didn't exc...
1,289689,827738,2012-01-02,358296,Alan,Staying at Christine's place was a great exper...
2,289689,1021213,2012-03-19,1090895,Lisa,We stayed at Killians apartment for five night...
3,289689,1098208,2012-04-08,737040,Brian,"Christine was very helpful, her place is in a ..."
4,289689,1291346,2012-05-16,2244965,Haus,"Hello, We made a round trip out of work and Fu..."


In [ ]:
df['date'] = pd.to_datetime(df['date'])
period1 = df[(df['date'] >= '2014-05-01') & (df['date'] <= '2016-12-31')]
period2 = df[(df['date'] >= '2017-01-01') & (df['date'] <= '2018-05-31')]

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    if isinstance(text, str):
        tokens = word_tokenize(text.lower())  # Tokenize and convert to lower case
        tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation
        tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
        return ' '.join(tokens)
    else:
        return ""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
period1['processed_comments'] = period1['comments'].apply(preprocess_text)
period2['processed_comments'] = period2['comments'].apply(preprocess_text)

<ipython-input-9-7c7c2001e11b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period1['processed_comments'] = period1['comments'].apply(preprocess_text)
<ipython-input-9-7c7c2001e11b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period2['processed_comments'] = period2['comments'].apply(preprocess_text)


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to get the compound sentiment score from a text
def get_sentiment(text):
    if text:
        return sia.polarity_scores(text)['compound']
    else:
        return 0


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
period1['sentiment_score'] = period1['processed_comments'].apply(get_sentiment)
period2['sentiment_score'] = period2['processed_comments'].apply(get_sentiment)

<ipython-input-11-7e91e25a5f8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period1['sentiment_score'] = period1['processed_comments'].apply(get_sentiment)
<ipython-input-11-7e91e25a5f8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period2['sentiment_score'] = period2['processed_comments'].apply(get_sentiment)


In [ ]:
average_sentiment_period1 = period1['sentiment_score'].mean()
average_sentiment_period2 = period2['sentiment_score'].mean()

In [ ]:
print("Average Sentiment Score for Period 1 (May 2014 - Dec 2016):", average_sentiment_period1)
print("Average Sentiment Score for Period 2 (Jan 2017 - May 2018):", average_sentiment_period2)


Average Sentiment Score for Period 1 (May 2014 - Dec 2016): 0.8673921901283222
Average Sentiment Score for Period 2 (Jan 2017 - May 2018): 0.8208318543480296


In [ ]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

# Helper function to prepare text data
def prepare_texts(texts):
    return [text.split() for text in texts if text]

# Prepare texts for each period
texts_period1 = prepare_texts(period1['processed_comments'])
texts_period2 = prepare_texts(period2['processed_comments'])

# Create a dictionary and corpus for each period
dictionary_period1 = corpora.Dictionary(texts_period1)
corpus_period1 = [dictionary_period1.doc2bow(text) for text in texts_period1]

dictionary_period2 = corpora.Dictionary(texts_period2)
corpus_period2 = [dictionary_period2.doc2bow(text) for text in texts_period2]


In [ ]:
# Number of topics
num_topics = 10

# Train LDA model for Period 1
lda_model_period1 = LdaModel(corpus_period1, num_topics=num_topics, id2word=dictionary_period1, passes=15)

# Train LDA model for Period 2
lda_model_period2 = LdaModel(corpus_period2, num_topics=num_topics, id2word=dictionary_period2, passes=15)


In [ ]:
topics_period1 = lda_model_period1.print_topics(num_words=5)
print("\nTopics from Period 1 (May 2014 - Dec 2016):")
for topic in topics_period1:
    print(topic)

topics_period2 = lda_model_period2.print_topics(num_words=5)
print("\nTopics from Period 2 (Jan 2017 - May 2018):")
for topic in topics_period2:
    print(topic)


Topics from Period 1 (May 2014 - Dec 2016):
(0, '0.043*"restaurants" + 0.035*"walking" + 0.029*"park" + 0.029*"distance" + 0.024*"within"')
(1, '0.031*"gavin" + 0.012*"damen" + 0.012*"responses" + 0.012*"per" + 0.010*"di"')
(2, '0.039*"de" + 0.029*"la" + 0.021*"et" + 0.018*"en" + 0.014*"un"')
(3, '0.034*"home" + 0.024*"us" + 0.017*"chicago" + 0.017*"stay" + 0.015*"house"')
(4, '0.055*"line" + 0.034*"walk" + 0.025*"blue" + 0.023*"downtown" + 0.021*"away"')
(5, '0.027*"und" + 0.018*"die" + 0.015*"chantell" + 0.013*"sehr" + 0.013*"man"')
(6, '0.030*"great" + 0.028*"place" + 0.022*"stay" + 0.020*"us" + 0.020*"nice"')
(7, '0.017*"room" + 0.014*"kitchen" + 0.012*"bed" + 0.011*"bathroom" + 0.010*"one"')
(8, '0.052*"nick" + 0.019*"steve" + 0.015*"staff" + 0.013*"cable" + 0.010*"ear"')
(9, '0.046*"great" + 0.037*"stay" + 0.036*"place" + 0.023*"location" + 0.023*"would"')

Topics from Period 2 (Jan 2017 - May 2018):
(0, '0.023*"room" + 0.023*"place" + 0.022*"nice" + 0.020*"good" + 0.018*"parkin

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus_pre_policy = period1['processed_comments'].tolist()
corpus_post_policy = period2['processed_comments'].tolist()

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust `max_features` as needed

# Fit and transform the corpora
tfidf_pre = tfidf_vectorizer.fit_transform(corpus_pre_policy)
tfidf_post = tfidf_vectorizer.fit_transform(corpus_post_policy)

# Getting feature names to understand what each column in the matrix represents
feature_names_pre = tfidf_vectorizer.get_feature_names_out()
feature_names_post = tfidf_vectorizer.get_feature_names_out()

In [ ]:
df_tfidf_pre = pd.DataFrame(tfidf_pre.toarray(), columns=feature_names_pre)
df_tfidf_post = pd.DataFrame(tfidf_post.toarray(), columns=feature_names_post)

# Calculate average TF-IDF score for each word
avg_scores_pre = df_tfidf_pre.mean(axis=0).sort_values(ascending=False)
avg_scores_post = df_tfidf_post.mean(axis=0).sort_values(ascending=False)

# Display top 10 terms from each period
print("Top 10 TF-IDF Terms in Pre-Policy Reviews:")
print(avg_scores_pre.head(10))

print("\nTop 10 TF-IDF Terms in Post-Policy Reviews:")
print(avg_scores_post.head(10))

Top 10 TF-IDF Terms in Pre-Policy Reviews:
highly       0.110517
place        0.091942
space        0.088671
apartment    0.068356
check        0.068111
trip         0.064969
house        0.060018
city         0.059380
nice         0.058402
lots         0.057669
dtype: float64

Top 10 TF-IDF Terms in Post-Policy Reviews:
great        0.123881
place        0.111647
stay         0.090009
location     0.068746
clean        0.062260
host         0.056523
nice         0.055704
chicago      0.054921
would        0.053091
apartment    0.048542
dtype: float64


In [ ]:
from textblob import TextBlob

# Function to extract sentiment around a specific word
def sentiment_around_word(text, word):
    sentences = [sentence for sentence in text.split('.') if word in sentence]
    sentiments = [TextBlob(sentence).sentiment.polarity for sentence in sentences]
    return sentiments

# Applying the function to your DataFrame
df_post_policy['location_sentiments'] = df_post_policy['comments'].apply(lambda x: sentiment_around_word(x, 'location'))

# Analyze the results
average_location_sentiment = df_post_policy['location_sentiments'].mean()

print("Average Sentiment for 'location':", average_location_sentiment)

NameError: name 'df_post_policy' is not defined

In [ ]:
listing = pd.read_csv('listings 2.csv')

In [ ]:
listing.head(1)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1461451,https://www.airbnb.com/rooms/1461451,20231218032601,2023-12-18,city scrape,Rental unit in Chicago · ★4.60 · 1 bedroom · 1...,NaN,We are in the far north neighborhood of Chicag...,https://a0.muscache.com/pictures/da1a79e0-ac55...,2907254,...,4.8,4.52,4.65,R21000075752,f,15,0,4,11,1.49


In [ ]:
len(listing)

8949